In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import functions as fun # functions mostly written by Alexendre Perez (and some by me)
from nistats import thresholding
import numpy as np
import nibabel as nib
import nilearn
from nilearn import masking, plotting, image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
import shutil
import os
import scipy.stats as st

template = nilearn.datasets.load_mni152_template()
affine = template.affine

level = .05
height_control = 'fdr'
cluster_threshold = 1
sample_size=108
cut_coords=(0, 0, 0) # brain coordinates for visualization

save_figs = False
save_results = False

hypothesis = 1

## Pipelines
### Load pipelines spreadsheet

In [2]:
pipelines_file = '../data-narps/narps_pipelines.csv'
pipel_str = pd.read_csv(pipelines_file)
pd.set_option('display.max_rows', pipel_str.shape[0]+1)

# remove spaces
pipel_str = fun.df_strip(pipel_str)
pipel_str

,Unnamed: 0,NV_collection_string,TSc_SW,used_fmriprep_data,TSc_smoothing,TSc_testing
0,50GV,LNYOVSMV,FSL,Yes,5,GRTFWE
1,9Q6R,EDFRMJVJ,FSL,No,5,GRTFWE
2,O21U,QTIWKLGM,FSL,Yes,5,GRTFWE
3,U26C,LOMXBZWB,SPM12,Yes,5,GRTFWE
4,43FJ,CLTGPQPO,FSL,No,5,GRTFWE
5,C88N,ADFZYYLQ,SPM,Yes,8,GRTFWE
6,4TQ6,WBTVHMSS,FSL,Yes,6,Randomise
7,T54A,IZREEBAR,FSL,Yes,4,TFCE
8,2T6S,WYHPMBFA,SPM,Yes,8,None
9,L7J7,QDWZHJWT,SPM,Yes,6,GRTFWE


### Turning columns with multiple values into booleans

In [3]:
columns_to_be_booleaned = ['TSc_SW','used_fmriprep_data','TSc_testing']
numerical_columns = ['TSc_smoothing']
for column in columns_to_be_booleaned:
    unique_strs = pipel_str[column].unique()
    for unique_str in unique_strs:
        if unique_str != 'No':
            str_in_col = pipel_str[column].str.find(unique_str)
            new_col_name = column + '_' + unique_str
            pipel_str[new_col_name] = str_in_col + 1  
            numerical_columns.append(new_col_name)
            
pipel_num = pipel_str[numerical_columns]

SPM = pipel_num['TSc_SW_SPM'] - pipel_num['TSc_SW_SPM12'] - pipel_num['TSc_SW_SPM8'] - pipel_num['TSc_SW_SPM5']
pipel_num['TSc_SW_SPM'] = SPM
pipel_num

,TSc_smoothing,TSc_SW_FSL,TSc_SW_SPM12,TSc_SW_SPM,TSc_SW_nistats,TSc_SW_randomise,TSc_SW_AFNI,TSc_SW_Other,TSc_SW_SPM8,TSc_SW_PALM,...,TSc_testing_Randomise,TSc_testing_TFCE,TSc_testing_None,TSc_testing_FDR,TSc_testing_ClustSim,TSc_testing_SnPM,TSc_testing_ARI,TSc_testing_BSR,TSc_testing_PALM,TSc_testing_ETAC
0,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,8,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,4,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,8,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,6,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Visualize

### FWHM of smoothing kernel

In [4]:
font = {'size'   : 16}
matplotlib.rc('font', **font)

fig, ax = plt.subplots(figsize=(10,5))
plt.hist(pipel_num['TSc_smoothing'], bins=np.max(pipel_num['TSc_smoothing']))
plt.xlabel('FWHM of smoothing kernel')
plt.ylabel('Number of teams')
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.tight_layout()

if save_figs: plt.savefig('../figures/results3_histogram_of_FWHM_kernels.png')

### Software used to get results maps

In [5]:
labels = np.unique(pipel_str['TSc_SW'])
cols = ['TSc_SW_' + label for label in labels]
x_pos = np.arange(len(labels))

counts = [np.sum(pipel_num[col]) for col in cols]

order = np.argsort(counts)
counts.sort()
labels = np.array(labels)
labels = labels[order]


fig, ax = plt.subplots(figsize=(10,5))
plt.bar(x_pos, counts[::-1], align='center')
plt.xticks(x_pos, labels[::-1], rotation='vertical')
plt.xlabel('Analysis software')
plt.ylabel('Number of teams')
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.tight_layout()


if save_figs: plt.savefig('../figures/results3_bar_analysis_software.png')

#### Software used to test SPMs

In [6]:
labels = np.unique(pipel_str['TSc_testing'])
cols = ['TSc_testing_' + label for label in labels]
x_pos = np.arange(len(labels))

counts = [np.sum(pipel_num[col]) for col in cols]

order = np.argsort(counts)
counts.sort()
labels = np.array(labels)
labels = labels[order]


fig, ax = plt.subplots(figsize=(10,5))
plt.bar(x_pos, counts[::-1], align='center')
plt.xticks(x_pos, labels[::-1], rotation='vertical')
plt.xlabel('Testing method')
plt.ylabel('Number of teams')
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.tight_layout()


if save_figs: plt.savefig('../figures/results3_bar_test_software.png')

## Compare study results across pipelines

### Get lists of studies based on analysis software

In [7]:
files = pipel_str['NV_collection_string']
SPM12 = files[pipel_num['TSc_SW_SPM12']>0]
FSL = files[pipel_num['TSc_SW_FSL']>0]

### Load neuroimaging data

In [8]:
img_paths = fun.get_data_paths_from_orig('hypo%d_unthresh.nii.gz' %hypothesis)
img_paths_SPM12 = fun.get_path_list_from_team_list(SPM12, img_paths)
img_paths_FSL = fun.get_path_list_from_team_list(FSL, img_paths)

In [9]:
def one_sample_ttest(img_paths):
    order = 'C'
    for n, path in enumerate(img_paths):
        img = nilearn.image.load_img(path)
        img_resampled = image.resample_to_img(img, template)
        array = img_resampled.get_fdata()
        flat = array.ravel(order=order)
        flat = np.array(flat)
        flat = np.expand_dims(flat, axis=1)
        if n == 0:
            y = flat
        else:
            y = np.concatenate((y, flat), axis=1)
    y=y.T
    Y = np.matrix(y)
    Y = np.nan_to_num(Y, copy=False, nan=0)

    x = np.ones_like(y[:,0])
    X = np.matrix(x).T


    beta = ((X.T * X)**-1) * X.T * Y

    n = Y.shape[0]
    residual = np.array(Y - X*beta)
    SSE = np.squeeze(np.sum(np.power(residual,2), axis=0))
    SD = np.sqrt(SSE/(n-1))
    SE = SD/np.sqrt(n)

    Z = np.arctanh(beta)

    T = beta/SE
    T_arr = T

    SE_map = fun.flat_mat_to_nii(flat_mat=SE, ref_niimg=template, order=order)
    B_map = fun.flat_mat_to_nii(flat_mat=beta, ref_niimg=template, order=order)
    Z_map = fun.flat_mat_to_nii(flat_mat=Z, ref_niimg=template, order=order)
    T_map = fun.flat_mat_to_nii(flat_mat=T, ref_niimg=template, order=order)
    
    return T_map, SE_map, B_map, Z_map, T_arr, T

In [10]:
T_s, SE_s, B_s, Z_s, T_arr_s, T = one_sample_ttest(img_paths_SPM12)
T_f, SE_f, B_f, Z_f, T_arr_f, T = one_sample_ttest(img_paths_FSL)

### Correlate maps

In [11]:
# pearson's
rp_f_s = np.corrcoef(T_arr_s, T_arr_f)
rp2 = rp_f_s[0,1] * rp_f_s[0,1]

# spearman's
rs_f_s = st.spearmanr(a=T_arr_s, b=T_arr_f, nan_policy='omit', axis=1)[0]
rs2 = rs_f_s*rs_f_s
print (rp2, rs2)

0.32343391579380143 0.15624613771754325


## Visualize

In [12]:
fishers_maps = { 
    'T-map from studies using SPM 12' : T_s,
    'T-map from studies using FSL' : T_f,
    'SE-map from studies using SPM 12' : SE_s,
    'SE-map from studies using FSL' : SE_f,
    #'Difference map (FSL - SPM12)': diff_f_minus_s,
}
for name, img in fishers_maps.items():
    plotting.plot_stat_map(fun.mni_mask(img), title=name, cut_coords=cut_coords,
                          figure=plt.figure(figsize=(10,5)))
    
    if save_figs: plt.savefig('../figures/results3_hypo%d_%s.png'\
                                  %(hypothesis, name.replace(' ', '-')))
    if save_results: img.to_filename('../data-narps/results3_hypo%d_%s.nii.gz' \
                                  %(hypothesis, name.replace(' ', '-')))

## PLS